In [1]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [2]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [3]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 8e7b022e
{
    "uid": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "e93cc55f-e3cc-4020-aa2b-0711757c476f",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "1575b6cf-41d3-4996-8dcc-73045c1d9539"
}


In [4]:
task_name = "Jupyter 2D Indoor"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 38a37900
{
    "uid": "38a37900-3bd5-4f9e-81e9-5903e13a9699",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:43.407127",
    "config": {
        "task_type": "parent",
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-2d-indoor_cdcad001",
    "parent": null,
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter 2D Indoor",
    "dependent_on": null,
    "priority": 100
}


In [5]:
stl_files = ["cfdGeom.stl", "heatZone.__0.stl", "heatZone.__1.stl"]

for file in stl_files:
    stl_path = os.path.join(data_path, "indoor", file)
    file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/{file}/",
                                files=open(stl_path, 'r').read(), extra_headers={"content-type": "application/octet-stream"})
    print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl
UPLOADED FILE: foam/constant/triSurface/heatZone.__0.stl
UPLOADED FILE: foam/constant/triSurface/heatZone.__1.stl


In [6]:
write_mesh_path = os.path.join(data_path, "indoor", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "indoor", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: c57a1b6c
{
    "uid": "c57a1b6c-6c1e-4657-b81d-0b825bfdfd67",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:43.813041",
    "config": {
        "task_type": "magpy",
        "cmd": "cfd.io.tasks.write_mesh",
        "base_mesh": "dict data too large for JSON.",
        "snappyhex_mesh": "dict data too large for JSON.",
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-2d-indoor_cdcad001",
    "parent": {
        "uid": "38a37900-3bd5-4f9e-81e9-5903e13a9699",
        "name": "Jupyter 2D Indoor"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Actions",
    "dependent_on": null,
    "priority": 100
}
TASK CONFIGS FOR ACTION TASK: c57a1b6c
[
    {
        "uid": "f46620ec-1fa6-4523-bf09-53191bf2c326",
        "used": false,
        "created": "2020-10-12T11:16:43.882515",
        "started": null,
        "stopped": null

In [7]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk",
            "foamToSurface -constant surfaceMesh.obj",
            "!setSet -batch zones.setSet"
        ],
        "cpus": cpus,
        "case_dir": "foam"
    },
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: bb1b7d8b
{
    "uid": "bb1b7d8b-e66d-48bf-b25a-82a08fe6371b",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:44.332339",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk",
            "foamToSurface -constant surfaceMesh.obj",
            "!setSet -batch zones.setSet"
        ],
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-2d-indoor_cdcad001",
    "parent": {
        "uid": "38a37900-3bd5-4f9e-81e9-5903e13a9699",
        "name": "Jupyter 2D Indoor"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Mesh",
    "dependent_on": {
        "uid": 

In [8]:
payload_openfoam = {
    "name": "SimpleCase",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "buoyantSimpleFoam",
            "reconstructPar -noZero"
        ],
        "cpus": [
            2,
            2,
            1
        ],
        "iterations": {
            "init": 50
        },
        "case_dir": "foam"
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

of_task = client.create(f"/api/project/{project_id}/task/", payload_openfoam)
of_id = of_task.get("uid")

print(f"VWT TASK: {of_id[:8]}")
print(json.dumps(of_task, indent=4))

VWT TASK: 9e4505d9
{
    "uid": "9e4505d9-044b-4820-b8ba-a28912ec0506",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:16:44.602719",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "buoyantSimpleFoam",
            "reconstructPar -noZero"
        ],
        "cpus": [
            2,
            2,
            1
        ],
        "iterations": {
            "init": 50
        },
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-2d-indoor_cdcad001",
    "parent": {
        "uid": "38a37900-3bd5-4f9e-81e9-5903e13a9699",
        "name": "Jupyter 2D Indoor"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "SimpleCase",
    "dependent_on": {
        "uid": "bb1b7d8b-e66d-48bf-b25a-82a08fe6371b",
        "name": "Mesh"
    },
    "priority": 100
}


In [9]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")